# Epub Translator Colab Notebook

This notebook allows you to use the Epub Translator tool to quickly translate epub books into bilingual book. The tool is designed to maintain the original text format while providing a rough translation.

To support my work please consider make donation: [Via Paypal](https://paypal.me/duocnguyen)

## Setup

First, let's install the necessary dependencies and the Epub Translator tool.

In [ ]:
# Install Epub Translator
!curl -fsSL https://raw.githubusercontent.com/nguyenvanduocit/epubtrans/main/scripts/install_unix.sh | bash

# Verify installation
!epubtrans --version

## Set up Anthropic API Key

To use the translation feature, you need to set up your Anthropic API key. Input your 

In [ ]:
import os
from getpass import getpass
anthropic_key = getpass('Enter your Anthropic API key: ')
os.environ['ANTHROPIC_KEY'] = anthropic_key

## Upload EPUB File

1. Run the code block below.
2. Click on "Choose Files" when prompted.
3. Select your EPUB file.
4. Click "Upload" to complete the process.

In [ ]:
from google.colab import files
uploaded = files.upload()

epub_filename = next(iter(uploaded))
print(f"Uploaded file: {epub_filename}")

## Translate the Book

Now, let's go through the steps to translate the book.

### 1. Unpack the epub file

In [ ]:
!epubtrans unpack "{epub_filename}"
unpacked_dir = epub_filename.rsplit('.', 1)[0]

### 2. Clean up HTML files

Clean unwanted HTML code that might break the book.

In [ ]:
!epubtrans clean "{unpacked_dir}"

### 3. Mark content for translation

Find all text content in the book and mark which text should be translated or not.

In [ ]:
!epubtrans mark "{unpacked_dir}"

### 4. Translate marked content

Start translating the marked content. This will take a lot of time, be patient.

In [ ]:
!epubtrans translate "{unpacked_dir}" --source English --target Vietnamese

### 5. (Optional) Style Adjustment

This step allows you to modify the visibility of the original text:

- To fade the original text, enhancing the translated version's visibility, run this block.
- To completely hide the original text, add the `--hide=source` option.

In [ ]:
!epubtrans styling "{unpacked_dir}"

### 6. Package into a bilingual book

In [ ]:
!epubtrans pack "{unpacked_dir}"

## Download the Translated Book

After the translation process is complete, you can download the translated book using the following code:

In [ ]:
import os
import re
from IPython.display import FileLink, display

# Function to extract the number from the filename
def extract_number(filename):
    match = re.search(r'\((\d+)\)', filename)
    return int(match.group(1)) if match else 0

# Get all files in the current directory
epubFiles = [f for f in os.listdir('.') if f.startswith(f"{unpacked_dir}-bilangual") and f.endswith('.epub')]

if not epubFiles:
    print("No matching files found.")
else:
    # Sort files based on the number in parentheses (if present)
    latest_file = max(epubFiles, key=extract_number)

    print(f"Latest file: {latest_file}")
    files.download(latest_file)

## Important Notes

1. The quality of translation depends on the Anthropic API and may not be perfect for all types of content.
2. Large books may take a considerable amount of time to translate.
3. Make sure to keep your Anthropic API key confidential and do not share it publicly.
4. This notebook provides a basic workflow. For more advanced usage or troubleshooting, refer to the [Epub Translator GitHub repository](https://github.com/nguyenvanduocit/epubtrans).

Feel free to modify this notebook to suit your specific needs or to experiment with different options provided by the Epub Translator tool.